#Kelompok 6
Anggota :
1.   Fachri Najm Noer Kartiman 2106515
2.   M. Daffa Fadhillah 2100543
3.   Rachman Faiz Maulana 2106791
4.   Elsa Nabiilah 2108805
5.   M. Salman Kesuma


# Import Library

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
import pickle
import joblib


In [3]:
dfClass = pd.read_csv("https://raw.githubusercontent.com/fachrinnk4869/data-order/master/orders_class.txt", sep=';')
dfTrain = pd.read_csv("https://raw.githubusercontent.com/fachrinnk4869/data-order/master/orders_train.txt", sep=';')

# Praproses

### Normalisai tanggal string '?' diganti menjadi None

In [4]:
dfTrain["size"]= dfTrain["size"].str.upper()
dfTrain["deliveryDate"] =  dfTrain["deliveryDate"].replace({
    "?": None,
})
dfTrain["dateOfBirth"] =  dfTrain["dateOfBirth"].replace({
    "?": None,
})
dfTrain["deliveryDate"] = pd.to_datetime(dfTrain["deliveryDate"])
dfTrain["orderDate"] = pd.to_datetime(dfTrain["orderDate"])
dfTrain["creationDate"] = pd.to_datetime(dfTrain["creationDate"])
dfTrain['dateOfBirth'] = pd.to_datetime(dfTrain['dateOfBirth'], errors='coerce')

### Ubah di atribut size '+' menjadi tidak ada

In [5]:
dfTrain.size = dfTrain["size"].str.replace("+","")

C:\Temp\ipykernel_16976\1718384458.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfTrain.size = dfTrain["size"].str.replace("+","")


### Ubah atribut size menjadi XL, XXL, dll.

In [6]:
dfTrain["size"] =  dfTrain["size"].replace({
    "XS": int(44), 
    "M": int(49), 
    "L": int(51),
    "S": int(47),
    "XL": int(53),
    "XXL" : int(55),
    "XXXL": int(57),
    "UNSIZED": int(59),
})

dfTrain.size = dfTrain["size"].astype(int)

dfTrain.loc[dfTrain['size'] < 44, "realsize"] = 'UNSIZED'
dfTrain.loc[dfTrain['size'] > 58, "realsize"] = 'UNSIZED'
dfTrain.loc[dfTrain['size'] == 44, "realsize"] = 'XS'
dfTrain.loc[(dfTrain['size'] >= 45) & (dfTrain['size'] <= 48), "realsize"] = 'S'
dfTrain.loc[(dfTrain['size'] >= 49) & (dfTrain['size'] <= 50), "realsize"] = 'M'
dfTrain.loc[(dfTrain['size'] >= 51) & (dfTrain['size'] <= 52), "realsize"] = 'L'
dfTrain.loc[(dfTrain['size'] >= 53) & (dfTrain['size'] <= 54), "realsize"] = 'XL'
dfTrain.loc[(dfTrain['size'] >= 55) & (dfTrain['size'] <= 56), "realsize"] = 'XXL'
dfTrain.loc[(dfTrain['size'] >= 57) & (dfTrain['size'] <= 58), "realsize"] = 'XXXL'
dfTrain['size'] = dfTrain['realsize']
dfTrain = dfTrain.drop(['realsize'], axis=1)
dfTrain

,orderItemID,orderDate,deliveryDate,itemID,size,color,manufacturerID,price,customerID,salutation,dateOfBirth,state,creationDate,returnShipment
0,1,2012-04-01,2012-04-03,186,M,denim,25,69.90,794,Mrs,1965-01-06,Baden-Wuerttemberg,2011-04-25,0
1,2,2012-04-01,2012-04-03,71,UNSIZED,ocher,21,69.95,794,Mrs,1965-01-06,Baden-Wuerttemberg,2011-04-25,1
2,3,2012-04-01,2012-04-03,71,UNSIZED,curry,21,69.95,794,Mrs,1965-01-06,Baden-Wuerttemberg,2011-04-25,1
3,4,2012-04-02,NaT,22,M,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0
4,5,2012-04-02,1990-12-31,151,UNSIZED,black,53,29.90,825,Mrs,1964-07-11,Rhineland-Palatinate,2011-02-16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481087,481088,2013-03-31,2013-04-09,2561,UNSIZED,blue,1,139.90,7032,Mr,1962-12-17,Lower Saxony,2011-03-12,1
481088,481089,2013-03-31,2013-04-02,2669,UNSIZED,grey,18,130.00,86610,Mrs,NaT,Berlin,2012-03-24,0
481089,481090,2013-03-31,1990-12-31,2458,XXL,black,5,99.90,86611,Mr,1973-05-16,Berlin,2013-03-31,0
481090,481091,2013-03-31,1990-12-31,2659,XXL,black,49,99.90,86611,Mr,1973-05-16,Berlin,2013-03-31,0


## Smoothing Warna

In [7]:
dfTrain["color"] =  dfTrain["color"].replace({
  "dark denim":"black", #blue?
  "dark navy":"blue",
  "ash":"grey",
  "bordeaux":"red",
  "mahagoni":"red", #brown?
  "gold":"yellow",
  "dark oliv":"green",
  "striped":"other",
  "anthracite":"black", #grey?
  "antique pink":"red",
  "floral":"other", #?
  "baltic blue":"blue",
  "nature":"other", #?
  "ancient":"other", #?
  "curry":"yellow",
  "turquoise":"blue",
  "navy":"blue",
  "brown":"brown",
  "aubergine":"red", #brown?
  "mocca":"brown", #grey?
  "blau":"blue",
  "basalt":"grey",
  "azure":"blue",
  "coral":"red", #yellow?
  "pallid":"grey", #?
  "petrol":"blue",
  "silver":"grey",
  "habana":"red", #?
  "darkblue":"blue",
  "beige":"yellow", #brown/grey?
  "mint":"green",
  "khaki":"brown", #yellow?
  "hibiscus":"red",
  "orange":"red", #yellow?
  "yellow":"yellow",
  "black":"black",
  "blue":"blue",
  "purple":"red",
  "almond":"brown", #yellow?
  "red":"red",
  "berry":"red",
  "grey":"grey",
  "ocher":"brown",
  "avocado":"green",
  "magenta":"red",
  "olive":"green",
  "white":"white",
  "denim":"other", #?
  "pink":"red",
  "stained":"brown",
  "kanel":"brown", #red?
  "green":"green",
  "jade":"green",
  "aquamarine":"blue",
  "aqua":"blue",
  "ecru":"grey", #brown/yellow?
  "iron":"grey",
  "fuchsia":"red",
  "ingwer":"red", #brown?
  "cognac":"brown",
  "terracotta":"brown", #red?
  "apricot":"yellow",
  "graphite":"grey",
  "crimson":"red",
  "lemon":"yellow",
  "oliv":"green",
  "leopard":"yellow", #brown/other?
  "amethyst":"red",
  "aviator":"other", #?
  "bronze":"brown",
  "brwon":"brown",
  "caramel":"yellow",
  "champagner":"yellow", #grey?
  "cobalt blue":"blue",
  "copper coin":"brown",
  "cortina mocca":"brown",
  "creme":"yellow",
  "curled":"other", #?
  "currant purple":"red",
  "dark garnet":"black", #red?
  "dark grey":"grey",
  "ebony":"black",
  "ivory":"white",
  "mango":"yellow",
  "opal":"other", #kann alles sein
  "perlmutt":"other", #?
  "vanille":"white",
  "?":"other",
})

In [8]:
dfTrain['color'].unique()

array(['other', 'brown', 'yellow', 'green', 'black', 'red', 'blue',
       'grey', 'white'], dtype=object)

### Mengganti date NULL dengan rata - rata

In [9]:
dfTrain["dateOfBirth"].fillna(dfTrain["dateOfBirth"].mean(), inplace = True)
dfTrain["deliveryDate"].fillna(dfTrain["deliveryDate"].mean(), inplace = True)

### Menambah atribut baru dengan mengekstrak tanggal, bulan, tahun

In [10]:
# diffdate adalah deliverytime yakni sbustraksi dari deliveryDte dengan orderDate
dfTrain['diffDate'] = (dfTrain['deliveryDate'] - dfTrain['orderDate']).dt.days
# accountAge(umur akun) = tahun dari deliveryDate dikurang tahun akun dibuat
dfTrain['accountAge'] = (dfTrain['deliveryDate'].dt.year - dfTrain['creationDate'].dt.year).astype(int)
# age(umur) = tahun dibuat = tahun lahir
dfTrain['age'] = (dfTrain['creationDate'].dt.year - dfTrain['dateOfBirth'].dt.year).astype(int)
# bulan berapa cutomer memesan
dfTrain['monthOrder'] = dfTrain['orderDate'].dt.month
# bulan berapa barang sampai ke customer
dfTrain['monthDelivery'] = dfTrain['deliveryDate'].dt.month
# hari apa cutomer memesan
dfTrain['dayOrder'] = dfTrain['orderDate'].dt.day
# hari apa barang sampai ke customer
dfTrain['dayDelivery'] = dfTrain['deliveryDate'].dt.day
# tahun berapa cutomer memesan
dfTrain['yearOrder'] = dfTrain['orderDate'].dt.year
# tahun berapa barang sampai ke customer
dfTrain['yearDelivery'] = dfTrain['deliveryDate'].dt.year

# membagi moth order ke 4 berdasarkan perbulanan di europe
dfTrain.loc[(dfTrain['monthOrder'] >= 1) & (dfTrain['monthOrder'] <=3), "season"] = 'spring'
dfTrain.loc[(dfTrain['monthOrder'] >= 4) & (dfTrain['monthOrder'] <= 6), "season"] = 'summer'
dfTrain.loc[(dfTrain['monthOrder'] >= 7) & (dfTrain['monthOrder'] <= 9), "season"] = 'fall'
dfTrain.loc[(dfTrain['monthOrder'] >= 10) & (dfTrain['monthOrder'] <= 12), "season"] = 'winter'

### menambah atribut baru berdasarkan persentase barang dikembalikan dan tidak dikembalikan berdasarkan itemID, customerID, dan age

In [11]:
for i in dfTrain['itemID'].unique():
  dfTrain.loc[(dfTrain['itemID'] == i), "percentItemReturn"] = len(dfTrain[(dfTrain['returnShipment'] == 1) & (dfTrain['itemID'] == i)]) / len(dfTrain[dfTrain['itemID'] == i])

for i in dfTrain['itemID'].unique():
  dfTrain.loc[(dfTrain['itemID'] == i), "percentItemNoReturn"] = len(dfTrain[(dfTrain['returnShipment'] == 0) & (dfTrain['itemID'] == i)]) / len(dfTrain[dfTrain['itemID'] == i])

for i in dfTrain['customerID'].unique():
  dfTrain.loc[(dfTrain['customerID'] == i), "percentCustReturn"] = len(dfTrain[(dfTrain['returnShipment'] == 1) & (dfTrain['customerID'] == i)]) / len(dfTrain[dfTrain['customerID'] == i])

for i in dfTrain['customerID'].unique():
  dfTrain.loc[(dfTrain['customerID'] == i), "percentCustNoReturn"] = len(dfTrain[(dfTrain['returnShipment'] == 0) & (dfTrain['customerID'] == i)]) / len(dfTrain[dfTrain['customerID'] == i])

for i in dfTrain['age'].unique():
  dfTrain.loc[(dfTrain['age'] == i), "percentAgeReturn"] = len(dfTrain[(dfTrain['returnShipment'] == 1) & (dfTrain['age'] == i)]) / len(dfTrain[dfTrain['age'] == i])

for i in dfTrain['age'].unique():
  dfTrain.loc[(dfTrain['age'] == i), "percentAgeNoReturn"] = len(dfTrain[(dfTrain['returnShipment'] == 0) & (dfTrain['age'] == i)]) / len(dfTrain[dfTrain['age'] == i])

In [12]:
dfTrain

,orderItemID,orderDate,deliveryDate,itemID,size,color,manufacturerID,price,customerID,salutation,...,dayDelivery,yearOrder,yearDelivery,season,percentItemReturn,percentItemNoReturn,percentCustReturn,percentCustNoReturn,percentAgeReturn,percentAgeNoReturn
0,1,2012-04-01,2012-04-03 00:00:00.000000000,186,M,other,25,69.90,794,Mrs,...,3,2012,2012,summer,0.643087,0.356913,0.454545,0.545455,0.494230,0.505770
1,2,2012-04-01,2012-04-03 00:00:00.000000000,71,UNSIZED,brown,21,69.95,794,Mrs,...,3,2012,2012,summer,0.610020,0.389980,0.454545,0.545455,0.494230,0.505770
2,3,2012-04-01,2012-04-03 00:00:00.000000000,71,UNSIZED,yellow,21,69.95,794,Mrs,...,3,2012,2012,summer,0.610020,0.389980,0.454545,0.545455,0.494230,0.505770
3,4,2012-04-02,2012-08-01 03:44:53.660694528,22,M,green,14,39.90,808,Mrs,...,1,2012,2012,summer,0.447011,0.552989,0.000000,1.000000,0.439690,0.560310
4,5,2012-04-02,1990-12-31 00:00:00.000000000,151,UNSIZED,black,53,29.90,825,Mrs,...,31,2012,1990,summer,0.588235,0.411765,0.111111,0.888889,0.482314,0.517686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481087,481088,2013-03-31,2013-04-09 00:00:00.000000000,2561,UNSIZED,blue,1,139.90,7032,Mr,...,9,2013,2013,spring,0.661290,0.338710,0.762712,0.237288,0.481299,0.518701
481088,481089,2013-03-31,2013-04-02 00:00:00.000000000,2669,UNSIZED,grey,18,130.00,86610,Mrs,...,2,2013,2013,spring,0.616216,0.383784,0.000000,1.000000,0.474917,0.525083
481089,481090,2013-03-31,1990-12-31 00:00:00.000000000,2458,XXL,black,5,99.90,86611,Mr,...,31,2013,1990,spring,0.636364,0.363636,0.000000,1.000000,0.506850,0.493150
481090,481091,2013-03-31,1990-12-31 00:00:00.000000000,2659,XXL,black,49,99.90,86611,Mr,...,31,2013,1990,spring,0.691057,0.308943,0.000000,1.000000,0.506850,0.493150


### Penanganan data yang kurang dari 0

In [13]:
#jika kurang dari 0 maka rubah data menjadi data yang sering muncul
dfTrain.loc[dfTrain['diffDate'] < 0, 'diffDate'] = dfTrain['diffDate'].mode()[0] 
#jika umur akun kurang dari 0 maka rubah data menjadi 0
dfTrain.loc[dfTrain['accountAge'] < 0, 'accountAge'] = 0 

### Drop Atribute yang tidak diperlukan

In [14]:
dfTrain = dfTrain.drop(['orderDate', 'deliveryDate','dateOfBirth', 'creationDate'], axis=1)


### Merubah beberapa column ke type data category

In [15]:
dfTrain['size'] = dfTrain['size'].astype('category')
dfTrain['color'] = dfTrain['color'].astype('category')
dfTrain['salutation'] = dfTrain['salutation'].astype('category')
dfTrain['state'] = dfTrain['state'].astype('category')
dfTrain['season'] = dfTrain['season'].astype('category')

### One hot encoder untuk atribut bertipe category

In [16]:
dfTrain = pd.get_dummies(data=dfTrain, columns=['size', 'color', 'salutation', 'state', 'season'])

# Melatih Data

In [17]:
dfTrain

,orderItemID,itemID,manufacturerID,price,customerID,returnShipment,diffDate,accountAge,age,monthOrder,...,state_Rhineland-Palatinate,state_Saarland,state_Saxony,state_Saxony-Anhalt,state_Schleswig-Holstein,state_Thuringia,season_fall,season_spring,season_summer,season_winter
0,1,186,25,69.90,794,0,2,1,46,4,...,0,0,0,0,0,0,0,0,1,0
1,2,71,21,69.95,794,1,2,1,46,4,...,0,0,0,0,0,0,0,0,1,0
2,3,71,21,69.95,794,1,2,1,46,4,...,0,0,0,0,0,0,0,0,1,0
3,4,22,14,39.90,808,0,121,0,53,4,...,0,0,1,0,0,0,0,0,1,0
4,5,151,53,29.90,825,0,2,0,47,4,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481087,481088,2561,1,139.90,7032,1,9,2,49,3,...,0,0,0,0,0,0,0,1,0,0
481088,481089,2669,18,130.00,86610,0,2,1,48,3,...,0,0,0,0,0,0,0,1,0,0
481089,481090,2458,5,99.90,86611,0,2,0,40,3,...,0,0,0,0,0,0,0,1,0,0
481090,481091,2659,49,99.90,86611,0,2,0,40,3,...,0,0,0,0,0,0,0,1,0,0


In [18]:
target = preprocessing.LabelEncoder()
target.fit(dfTrain.returnShipment)
Y = target.transform(dfTrain.returnShipment)

In [19]:
X = dfTrain.drop("returnShipment",axis=1)

In [20]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=123)
with open('xTrainColumn.pickle', 'wb') as fp:
    pickle.dump(X_train.columns, fp)


In [21]:
clf = RandomForestClassifier(n_estimators=200, random_state=123)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Akurasi 0.756991862314096
[[36634 13089]
 [10293 36203]]
              precision    recall  f1-score   support

           0       0.78      0.74      0.76     49723
           1       0.73      0.78      0.76     46496

    accuracy                           0.76     96219
   macro avg       0.76      0.76      0.76     96219
weighted avg       0.76      0.76      0.76     96219



In [22]:
# clf = xgb.XGBClassifier(objective = "multi:softprob", num_class = 2, eval_metric = "mlogloss", max_depth = 24, gamma=0.1, subsample = 0.90,learning_rate=0.01, n_estimators = 10, nthread=-1)
# # num_class adalah jumlah kelas
# clf.fit(X_train, Y_train)
# Y_pred = clf.predict(X_test)
# acc = accuracy_score(Y_test, Y_pred)
# print("Akurasi {}".format(acc))
# print(classification_report(Y_test, Y_pred))

In [23]:
feature_importances = pd.DataFrame({
    'attr' : X_train.columns, 
    'importance' :clf.feature_importances_
}).sort_values('importance',ascending=False)
# pd.set_option('display.max_rows', None)
feature_importances

,attr,importance
16,percentCustReturn,0.154751
17,percentCustNoReturn,0.153743
11,dayDelivery,0.065895
0,orderItemID,0.062307
14,percentItemReturn,0.055426
...,...,...
53,state_Saarland,0.000828
38,salutation_Family,0.000301
27,size_XXXL,0.000111
37,salutation_Company,0.000083


# Prediksi dataset baru

## Menyimpan Model

In [24]:
import joblib
joblib.dump(clf, "modelTrain.joblib")

['modelTrain.joblib']

# Praproses

### Normalisai tanggal string '?' diganti menjadi None

In [25]:
dfClass["size"]= dfClass["size"].str.upper()
dfClass["deliveryDate"] =  dfClass["deliveryDate"].replace({
    "?": None,
})
dfClass["dateOfBirth"] =  dfClass["dateOfBirth"].replace({
    "?": None,
})
dfClass["deliveryDate"] = pd.to_datetime(dfClass["deliveryDate"])
dfClass["orderDate"] = pd.to_datetime(dfClass["orderDate"])
dfClass["creationDate"] = pd.to_datetime(dfClass["creationDate"])
dfClass['dateOfBirth'] = pd.to_datetime(dfClass['dateOfBirth'], errors='coerce')

### Ubah di atribut size '+' menjadi tidak ada

In [26]:
dfClass.size = dfClass["size"].str.replace("+","")

C:\Temp\ipykernel_16976\3276738391.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfClass.size = dfClass["size"].str.replace("+","")


### Ubah atribut size menjadi XL, XXL, dll.

In [27]:
dfClass["size"] =  dfClass["size"].replace({
    "XS": int(44), 
    "M": int(49), 
    "L": int(51),
    "S": int(47),
    "XL": int(53),
    "XXL" : int(55),
    "XXXL": int(57),
    "UNSIZED": int(59),
})

dfClass.size = dfClass["size"].astype(int)

dfClass.loc[dfClass['size'] < 44, "realsize"] = 'UNSIZED'
dfClass.loc[dfClass['size'] > 58, "realsize"] = 'UNSIZED'
dfClass.loc[dfClass['size'] == 44, "realsize"] = 'XS'
dfClass.loc[(dfClass['size'] >= 45) & (dfClass['size'] <= 48), "realsize"] = 'S'
dfClass.loc[(dfClass['size'] >= 49) & (dfClass['size'] <= 50), "realsize"] = 'M'
dfClass.loc[(dfClass['size'] >= 51) & (dfClass['size'] <= 52), "realsize"] = 'L'
dfClass.loc[(dfClass['size'] >= 53) & (dfClass['size'] <= 54), "realsize"] = 'XL'
dfClass.loc[(dfClass['size'] >= 55) & (dfClass['size'] <= 56), "realsize"] = 'XXL'
dfClass.loc[(dfClass['size'] >= 57) & (dfClass['size'] <= 58), "realsize"] = 'XXXL'
dfClass['size'] = dfClass['realsize']
dfClass = dfClass.drop(['realsize'], axis=1)
dfClass

,orderItemID,orderDate,deliveryDate,itemID,size,color,manufacturerID,price,customerID,salutation,dateOfBirth,state,creationDate
0,1,2013-04-01,2013-04-03,2347,UNSIZED,magenta,1,89.9,12489,Mrs,1963-04-26,Hesse,2012-04-23
1,2,2013-04-01,2013-04-03,2741,UNSIZED,grey,1,99.9,12489,Mrs,1963-04-26,Hesse,2012-04-23
2,3,2013-04-01,2013-04-03,2514,UNSIZED,ecru,19,79.9,12489,Mrs,1963-04-26,Hesse,2012-04-23
3,4,2013-04-01,2013-05-06,2347,UNSIZED,brown,1,89.9,12489,Mrs,1963-04-26,Hesse,2012-04-23
4,5,2013-04-01,NaT,2690,UNSIZED,grey,1,119.9,12489,Mrs,1963-04-26,Hesse,2012-04-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50073,50074,2013-04-29,2013-05-03,2342,M,terracotta,5,69.9,91920,Mrs,1962-03-08,Bavaria,2013-04-29
50074,50075,2013-04-29,2013-05-03,2505,M,terracotta,5,64.9,91920,Mrs,1962-03-08,Bavaria,2013-04-29
50075,50076,2013-04-28,2013-05-02,2470,XL,white,5,79.9,85095,Mrs,1950-02-14,Berlin,2013-03-24
50076,50077,2013-04-28,NaT,2452,L,white,5,59.9,91922,Mrs,1969-11-27,Brandenburg,2013-04-28


## Smoothing Warna

In [28]:
dfClass["color"] =  dfClass["color"].replace({
  "dark denim":"black", #blue?
  "dark navy":"blue",
  "ash":"grey",
  "bordeaux":"red",
  "mahagoni":"red", #brown?
  "gold":"yellow",
  "dark oliv":"green",
  "striped":"other",
  "anthracite":"black", #grey?
  "antique pink":"red",
  "floral":"other", #?
  "baltic blue":"blue",
  "nature":"other", #?
  "ancient":"other", #?
  "curry":"yellow",
  "turquoise":"blue",
  "navy":"blue",
  "brown":"brown",
  "aubergine":"red", #brown?
  "mocca":"brown", #grey?
  "blau":"blue",
  "basalt":"grey",
  "azure":"blue",
  "coral":"red", #yellow?
  "pallid":"grey", #?
  "petrol":"blue",
  "silver":"grey",
  "habana":"red", #?
  "darkblue":"blue",
  "beige":"yellow", #brown/grey?
  "mint":"green",
  "khaki":"brown", #yellow?
  "hibiscus":"red",
  "orange":"red", #yellow?
  "yellow":"yellow",
  "black":"black",
  "blue":"blue",
  "purple":"red",
  "almond":"brown", #yellow?
  "red":"red",
  "berry":"red",
  "grey":"grey",
  "ocher":"brown",
  "avocado":"green",
  "magenta":"red",
  "olive":"green",
  "white":"white",
  "denim":"other", #?
  "pink":"red",
  "stained":"brown",
  "kanel":"brown", #red?
  "green":"green",
  "jade":"green",
  "aquamarine":"blue",
  "aqua":"blue",
  "ecru":"grey", #brown/yellow?
  "iron":"grey",
  "fuchsia":"red",
  "ingwer":"red", #brown?
  "cognac":"brown",
  "terracotta":"brown", #red?
  "apricot":"yellow",
  "graphite":"grey",
  "crimson":"red",
  "lemon":"yellow",
  "oliv":"green",
  "leopard":"yellow", #brown/other?
  "amethyst":"red",
  "aviator":"other", #?
  "bronze":"brown",
  "brwon":"brown",
  "caramel":"yellow",
  "champagner":"yellow", #grey?
  "cobalt blue":"blue",
  "copper coin":"brown",
  "cortina mocca":"brown",
  "creme":"yellow",
  "curled":"other", #?
  "currant purple":"red",
  "dark garnet":"black", #red?
  "dark grey":"grey",
  "ebony":"black",
  "ivory":"white",
  "mango":"yellow",
  "opal":"other", #kann alles sein
  "perlmutt":"other", #?
  "vanille":"white",
  "?":"other",
})

In [29]:
dfClass['color'].unique()

array(['red', 'grey', 'brown', 'blue', 'white', 'green', 'black',
       'yellow', 'other'], dtype=object)

### Mengganti date NULL dengan rata - rata

In [30]:
dfClass["dateOfBirth"].fillna(dfClass["dateOfBirth"].mean(), inplace = True)
dfClass["deliveryDate"].fillna(dfClass["deliveryDate"].mean(), inplace = True)

### Menambah atribut baru dengan mengekstrak tanggal, bulan, tahun

In [31]:
# diffdate adalah deliverytime yakni sbustraksi dari deliveryDte dengan orderDate
dfClass['diffDate'] = (dfClass['deliveryDate'] - dfClass['orderDate']).dt.days
# accountAge(umur akun) = tahun dari deliveryDate dikurang tahun akun dibuat
dfClass['accountAge'] = (dfClass['deliveryDate'].dt.year - dfClass['creationDate'].dt.year).astype(int)
# age(umur) = tahun dibuat = tahun lahir
dfClass['age'] = (dfClass['creationDate'].dt.year - dfClass['dateOfBirth'].dt.year).astype(int)
# bulan berapa cutomer memesan
dfClass['monthOrder'] = dfClass['orderDate'].dt.month
# bulan berapa barang sampai ke customer
dfClass['monthDelivery'] = dfClass['deliveryDate'].dt.month
# hari apa cutomer memesan
dfClass['dayOrder'] = dfClass['orderDate'].dt.day
# hari apa barang sampai ke customer
dfClass['dayDelivery'] = dfClass['deliveryDate'].dt.day
# tahun berapa cutomer memesan
dfClass['yearOrder'] = dfClass['orderDate'].dt.year
# tahun berapa barang sampai ke customer
dfClass['yearDelivery'] = dfClass['deliveryDate'].dt.year

# membagi moth order ke 4 berdasarkan perbulanan di europe
dfClass.loc[(dfClass['monthOrder'] >= 1) & (dfClass['monthOrder'] <=3), "season"] = 'spring'
dfClass.loc[(dfClass['monthOrder'] >= 4) & (dfClass['monthOrder'] <= 6), "season"] = 'summer'
dfClass.loc[(dfClass['monthOrder'] >= 7) & (dfClass['monthOrder'] <= 9), "season"] = 'fall'
dfClass.loc[(dfClass['monthOrder'] >= 10) & (dfClass['monthOrder'] <= 12), "season"] = 'winter'

### menambah atribut baru berdasarkan persentase barang dikembalikan dan tidak dikembalikan berdasarkan itemID, customerID, dan age

In [32]:
for i in dfTrain['itemID'].unique():
  dfClass.loc[(dfClass['itemID'] == i), "percentItemReturn"] = dfTrain[(dfTrain['itemID'] == i)]['percentItemReturn'].values[0]

for i in dfTrain['itemID'].unique():
  dfClass.loc[(dfClass['itemID'] == i), "percentItemNoReturn"] = dfTrain[(dfTrain['itemID'] == i)]['percentItemNoReturn'].values[0]

for i in dfTrain['customerID'].unique():
  dfClass.loc[(dfClass['customerID'] == i), "percentCustReturn"] = dfTrain[(dfTrain['customerID'] == i)]['percentCustReturn'].values[0]

for i in dfTrain['customerID'].unique():
  dfClass.loc[(dfClass['customerID'] == i), "percentCustNoReturn"] = dfTrain[(dfTrain['customerID'] == i)]['percentCustNoReturn'].values[0]

for i in dfTrain['age'].unique():
  dfClass.loc[(dfClass['age'] == i), "percentAgeReturn"] = dfTrain[(dfTrain['age'] == i)]['percentAgeReturn'].values[0]

for i in dfTrain['age'].unique():
  dfClass.loc[(dfClass['age'] == i), "percentAgeNoReturn"] = dfTrain[(dfTrain['age'] == i)]['percentAgeNoReturn'].values[0]

dfClass["percentItemReturn"].fillna(0, inplace = True)
dfClass["percentItemNoReturn"].fillna(0, inplace = True)
dfClass["percentCustReturn"].fillna(0, inplace = True)
dfClass["percentCustNoReturn"].fillna(0, inplace = True)
dfClass["percentAgeReturn"].fillna(0, inplace = True)
dfClass["percentAgeNoReturn"].fillna(0, inplace = True)
# dfTrain[(dfTrain['customerID'] == 794)]['percentCustReturn'].values[0]

In [33]:
dfClass

,orderItemID,orderDate,deliveryDate,itemID,size,color,manufacturerID,price,customerID,salutation,...,dayDelivery,yearOrder,yearDelivery,season,percentItemReturn,percentItemNoReturn,percentCustReturn,percentCustNoReturn,percentAgeReturn,percentAgeNoReturn
0,1,2013-04-01,2013-04-03 00:00:00.000000000,2347,UNSIZED,red,1,89.9,12489,Mrs,...,3,2013,2013,summer,0.458175,0.541825,0.333333,0.666667,0.481299,0.518701
1,2,2013-04-01,2013-04-03 00:00:00.000000000,2741,UNSIZED,grey,1,99.9,12489,Mrs,...,3,2013,2013,summer,0.604938,0.395062,0.333333,0.666667,0.481299,0.518701
2,3,2013-04-01,2013-04-03 00:00:00.000000000,2514,UNSIZED,grey,19,79.9,12489,Mrs,...,3,2013,2013,summer,0.654255,0.345745,0.333333,0.666667,0.481299,0.518701
3,4,2013-04-01,2013-05-06 00:00:00.000000000,2347,UNSIZED,brown,1,89.9,12489,Mrs,...,6,2013,2013,summer,0.458175,0.541825,0.333333,0.666667,0.481299,0.518701
4,5,2013-04-01,2013-01-26 15:35:49.626718976,2690,UNSIZED,grey,1,119.9,12489,Mrs,...,26,2013,2013,summer,0.525641,0.474359,0.333333,0.666667,0.481299,0.518701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50073,50074,2013-04-29,2013-05-03 00:00:00.000000000,2342,M,brown,5,69.9,91920,Mrs,...,3,2013,2013,summer,0.485429,0.514571,0.000000,0.000000,0.459081,0.540919
50074,50075,2013-04-29,2013-05-03 00:00:00.000000000,2505,M,brown,5,64.9,91920,Mrs,...,3,2013,2013,summer,0.589286,0.410714,0.000000,0.000000,0.459081,0.540919
50075,50076,2013-04-28,2013-05-02 00:00:00.000000000,2470,XL,white,5,79.9,85095,Mrs,...,2,2013,2013,summer,0.585567,0.414433,1.000000,0.000000,0.428288,0.571712
50076,50077,2013-04-28,2013-01-26 15:35:49.626718976,2452,L,white,5,59.9,91922,Mrs,...,26,2013,2013,summer,0.425718,0.574282,0.000000,0.000000,0.507490,0.492510


### Penanganan data yang kurang dari 0

In [34]:
#jika kurang dari 0 maka rubah data menjadi data yang sering muncul
dfClass.loc[dfClass['diffDate'] < 0, 'diffDate'] = dfClass['diffDate'].mode()[0] 
#jika umur akun kurang dari 0 maka rubah data menjadi 0
dfClass.loc[dfClass['accountAge'] < 0, 'accountAge'] = 0 

### Drop Atribute yang tidak diperlukan

In [35]:
dfClass = dfClass.drop(['orderDate', 'deliveryDate','dateOfBirth', 'creationDate'], axis=1)


### Merubah beberapa column ke type data category

In [36]:
dfClass['size'] = dfClass['size'].astype('category')
dfClass['color'] = dfClass['color'].astype('category')
dfClass['salutation'] = dfClass['salutation'].astype('category')
dfClass['state'] = dfClass['state'].astype('category')
dfClass['season'] = dfClass['season'].astype('category')
dfClass

,orderItemID,itemID,size,color,manufacturerID,price,customerID,salutation,state,diffDate,...,dayDelivery,yearOrder,yearDelivery,season,percentItemReturn,percentItemNoReturn,percentCustReturn,percentCustNoReturn,percentAgeReturn,percentAgeNoReturn
0,1,2347,UNSIZED,red,1,89.9,12489,Mrs,Hesse,2,...,3,2013,2013,summer,0.458175,0.541825,0.333333,0.666667,0.481299,0.518701
1,2,2741,UNSIZED,grey,1,99.9,12489,Mrs,Hesse,2,...,3,2013,2013,summer,0.604938,0.395062,0.333333,0.666667,0.481299,0.518701
2,3,2514,UNSIZED,grey,19,79.9,12489,Mrs,Hesse,2,...,3,2013,2013,summer,0.654255,0.345745,0.333333,0.666667,0.481299,0.518701
3,4,2347,UNSIZED,brown,1,89.9,12489,Mrs,Hesse,35,...,6,2013,2013,summer,0.458175,0.541825,0.333333,0.666667,0.481299,0.518701
4,5,2690,UNSIZED,grey,1,119.9,12489,Mrs,Hesse,2,...,26,2013,2013,summer,0.525641,0.474359,0.333333,0.666667,0.481299,0.518701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50073,50074,2342,M,brown,5,69.9,91920,Mrs,Bavaria,4,...,3,2013,2013,summer,0.485429,0.514571,0.000000,0.000000,0.459081,0.540919
50074,50075,2505,M,brown,5,64.9,91920,Mrs,Bavaria,4,...,3,2013,2013,summer,0.589286,0.410714,0.000000,0.000000,0.459081,0.540919
50075,50076,2470,XL,white,5,79.9,85095,Mrs,Berlin,4,...,2,2013,2013,summer,0.585567,0.414433,1.000000,0.000000,0.428288,0.571712
50076,50077,2452,L,white,5,59.9,91922,Mrs,Brandenburg,2,...,26,2013,2013,summer,0.425718,0.574282,0.000000,0.000000,0.507490,0.492510


### One hot encoder untuk atribut bertipe category

In [37]:
dfClass = pd.get_dummies(data=dfClass, columns=['size', 'color', 'salutation', 'state', 'season'])

# Prediksi data baru

In [38]:
dfClass['season_fall'] = 0
dfClass['season_spring'] = 0
dfClass['season_winter'] = 0

In [39]:
import pickle
with open ('xTrainColumn.pickle', 'rb') as fp:
    X_train_column = list(pickle.load(fp))
    
dfClass = dfClass[X_train_column]  

In [40]:
clf = joblib.load("modelTrain.joblib")
hasil = clf.predict(dfClass)
d = {'orderItemID': dfClass['orderItemID'].to_numpy().tolist(), 'prediction': hasil}
hasil = pd.DataFrame(data = d)
hasil

,orderItemID,prediction
0,1,0
1,2,0
2,3,1
3,4,0
4,5,0
...,...,...
50073,50074,0
50074,50075,0
50075,50076,1
50076,50077,0


In [43]:
hasil['prediction'].value_counts()

0    28747
1    21331
Name: prediction, dtype: int64

In [42]:
np.savetxt(r'hasil.txt', hasil , fmt='%d', delimiter=';',header="orderItemID;prediction", comments='')